# Lab 4 - Model Evaluation

Here, we will approach fundamental concepts of model evaluation: 

1. *Evaluation metrics*
2. *K-Fold cross validation*
3. *Experimental results* on best performance

----

```
Author:   Luis Quintero
Contact:  luis-eduardo@dsv.su.se | luisqtr.com
---
Notebook based on notes from Christian Kauth (UniFribourg), Jake VanderPlas (DataScienceHandbook), and Zed Lee (DSV,SU)
```

----

First, we import the main Python packages and initialize relevant variables

In [ ]:
# Numeric analysis
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt

In [ ]:
# Set the seed of the pseudo randomization to guarantee that results are reproducible between executions
RANDOM_SEED = 2023
np.random.seed(RANDOM_SEED)

# Loading and performing EDA in real-dataset

Let's bring again the *Bank Marketing* dataset available used in Lab 1 and available in the UCI repository: https://archive.ics.uci.edu/ml/datasets/Bank%2BMarketing

#### Input variables:

**bank client data:**

1. `age` (numeric)
2. `job` : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. `marital` : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. `education` (categorical: 'primary','secondary','terciary,'unknown')
5. `default`: has credit in default? (categorical: 'no','yes','unknown')
6. `balance`: average yearly balance, in euros (numeric) 
7. `housing`: has housing loan? (categorical: 'no','yes','unknown')
8. `loan`: has personal loan? (categorical: 'no','yes','unknown')

**related with the last contact of the current campaign:**

1. `contact`: contact communication type (categorical: 'cellular','telephone')
2.  `month`: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
3.  `day_of_week`: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
4.  `duration`: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

**other attributes:**

5.  `campaign`: number of contacts performed during this campaign and for this client (numeric, includes last contact)
6.  `pdays`: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
7.  `previous`: number of contacts performed before this campaign and for this client (numeric)
8.  `poutcome`: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

#### Output variable (desired target):

17. `y` - has the client subscribed a term deposit? (binary: 'yes','no')

In [ ]:
# Pandas provides built-in functions to load text files into DataFrames.
data = pd.read_csv("bank.csv", sep=";", true_values =["yes"], false_values=["no"])
data

In [ ]:
data.dtypes

In [ ]:
data.describe(include="all")

### Organizing data types

Above we are describing both numerical and non-numerical features. Therefore, some metrics don't apply for some columns. For example, it's impossible to calculate the `mean` of the feature `education` because it contains only text, not numbers.

However, the table is interesting to discover that non-numerical features like `job`, `marital`, `education`, `contact`, `month`, `poutcome` have few **unique** categories. Let's change the data types accordingly.

In [ ]:
### ORDERED CATEGORICAL FEATURES
# The months are incremental, therefore they have an order: 'jan'<'feb'<'mar'<...<'dec'.
print("List of values in the feature `month`:", data["month"].unique(), "and dtype=",data["month"].dtypes)

# We may consider `education` has a variable that has an order. People cannot get reach secondary level, without having primary education.
print("List of values in the feature `education`:", data["education"].unique(), "and dtype=",data["education"].dtypes)

In [ ]:
# We apply Ordinal Encoding, as in the example from Lab3
months_order = ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
data["month"] = pd.Categorical(data["month"], categories=months_order, ordered=True)

print("List of values in the feature `month`:", data["month"].unique(), "and dtype=",data["month"].dtypes)

# # Another option would be to map the month names to their corresponding numerical value.
# # but we will continue with `ordered categories`` for educational purposes
# data["month"] = data["month"].map({
#                                     "jan":1, "feb":2, "mar":3,
#                                     "apr":4, "may":5, "jun":6,
#                                     "jul":7, "aug":8, "sep":9,
#                                     "oct":10,"nov":11,"dic":12,
#                                 })

In [ ]:
# Ordinal encoding for `education`
education_order = ["primary", "secondary", "tertiary", "unknown"]
data["education"] = pd.Categorical(data["education"], categories=education_order, ordered=True)

print("List of values in the feature `education`:", data["education"].unique(), "and dtype=",data["education"].dtypes)

In [ ]:
# The rest of the categorical variables are unordered.

# NOTE: You can also access the column name just using `dot` between the name of the dataframe and the column.
# For example, the code to access the feature `data["job"]` can also be written as `data.job`

data.job = data.job.astype("category")
data.marital = data.marital.astype("category")
data.contact = data.contact.astype("category")
data.poutcome = data.poutcome.astype("category")

In [ ]:
data.dtypes

### Handling Missing Values and Filtering data

In [ ]:
# Check if there are missing values
data.isnull().sum()

## There are no missing values (all counts are zero) 
## If there were, you can delete rows with missing values, or impute values using
# data.dropna(inplace=True)
## Or impute values
# data["balance"].fillna(data["balance"].mean(), inplace=True)
# data["month"].fillna(data["month"].value_counts().idxmax(), inplace=True) # For categorical features


For simplicity, let's assume we have some understanding of the business problem and we have selected a subset of features from the whole dataset, which we think may be related to the outcome variable. In real data science projects, it is necessary to experiment with several combinations of features, or even conduct feature engineering to create new variables.

Here, we have deliberately chosen some features to have multiple data types and to train classification models: `age`, `marital`, `education`, `default`, `balance`, `loan`, `month`, `duration`, `campaign`, and ***obviously***, the target variable `y`.

- The numerical features are: `age`, `balance`, `duration`, `campaign`
- The categorical features are: `marital` (unordered), `education` (ordered), `month` (ordered)
- The binary features are: `default`, `loan`, and the target `y` (binary classification task)

### Conceptual Note❗❗

Every decision in the data science pipeline has an influence on the performance of the classification models. *E.g., the data imputation technique, the selection of features, which classifiers to train, which hyperparameters to use in each classifiers, which evaluation metric is needed to compare multiple classifiers.*

Therefore, note that here we are making a deliberate decision on the features to choose, which may lead to a very poor performance in the classifiers. The goal of the labs is to learn the tools and how to reason on the results, not to achieve the best classifier ever, so do not worry if in the lab or in your homework the performance is low, it will not determine the grading of the lab. Doing experiments to reach high classification performance is part of **data science research** and maybe of interest if you want to do a thesis in the area.

In [ ]:
# Select some of the existing features
data_filtered = data[["age","marital","education","default","balance","loan","month","duration","campaign","y"]]
data_filtered.describe(include="all")

In [ ]:
data_filtered["marital"].value_counts()

In [ ]:
data_filtered["education"].value_counts()

In [ ]:
data_filtered.shape

Let's also assume that we want to delete the 187 rows with value `unknown` in the `education` feature. 

In [ ]:
data_filtered = data_filtered[ (data_filtered["education"] != "unknown") ] # the operator `!=`` means "different than"

In [ ]:
data_filtered["education"].value_counts()

In [ ]:
data_filtered.shape
# The 187 samples were deleted

In [ ]:
# Plot class balance
data_filtered["y"].value_counts().plot.bar()

In [ ]:
# Data to be used for the classification task
data_filtered.sample(10)

Above, we can see that there is much more data from people who did not subscribe to the financial service `y=False`, than people who subscribed `y=True`.

Generally, we apply methods to **balance the classes**, but it is outside of the scope of the lab. We just need to keep in mind that certain metrics like `accuracy` are not sufficient to describe the performance of a classifier, and we will need to use other metrics such as `precision`, `recall`, and `f1-score`.

### Preparing dataframes for Classification Tasks

We have already loaded the dataset, conducted basic preprocessing and feature selection. The final step is to **transform the pandas DataFrame into numerical Numpy arrays**, so that they can be processed by the packages in `sklearn`. At this stage, we need to **encode the categorical features** as described in Lab3.

In [ ]:
# Apply Ordinal Encoding in ordered categories to get the numbers
labels_month, unique_month = pd.factorize(data_filtered["month"], sort=True)
data_filtered["month"] = labels_month

labels_edu, unique_edu = pd.factorize(data_filtered["education"], sort=True)
data_filtered["education"] = labels_edu

In [ ]:
# Apply one-hot encoding on unordered categories to get the columns
ohe_marital = pd.get_dummies(data_filtered["marital"])
data_filtered = pd.concat([data_filtered, ohe_marital], axis=1)
data_filtered = data_filtered.drop(["marital"], axis=1)

In [ ]:
# Convert to numeric the columns that are binary
data_filtered["default"] = data_filtered["default"].astype(int)
data_filtered["loan"] = data_filtered["loan"].astype(int)
data_filtered["y"] = data_filtered["y"].astype(int)

In [ ]:
data_filtered

In [ ]:
# Separate the features X and the target variable y
data_filtered_X = data_filtered.drop(["y"], axis=1)
data_filtered_y = data_filtered["y"]

In [ ]:
# Finally, we need transform from Pandas DataFrame to numerical Arrays, and store the column names
data_X = data_filtered_X.values
data_y = data_filtered_y.values

data_colnames = data_filtered_X.columns.values
print(data_colnames)

### Final feature matrix $\mathbf{X}$

In [ ]:
data_X

### Final target array $\mathbf{y}$

In [ ]:
data_y

# 1. Evaluation metrics

Officially we have finished with the preprocessing stage. Now, we have created the two inputs for any classification model ($\mathbf{X}, y$). The resulting post processed variables `(data_X, data_y)` are equivalent to the synthetic data obtained in previous labs with the functions `make_blobs()` or `make_moons()`. 

However, we are now using **real datasets** and **more than 2 features**, so we cannot visualize the classification performance by the plots that we had before.

The remaining of the lab presents the way to evaluate classification models when we cannot plot the decision boundaries, as in most cases with large datasets.

### Single train-test split

From the previous lab, we are using a single train-test split using the command below:

In [ ]:
from sklearn.model_selection import train_test_split

# 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size = 0.2, random_state=RANDOM_SEED)

In [ ]:
# An example of the classifier in the data
from sklearn.tree import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier(max_depth=10)
dt_classifier.fit(X_train, y_train)
y_predicted = dt_classifier.predict(X_test)
print(y_predicted)

So far, we had visualized the decision boundaries because we only had two variables in the **input feature matrix** $\mathbf{X}$. Bu we cannot plot the **9 features** in our new dataset.

Therefore, we use other proxy metrics to compare how the predicted labels compare with the true labels. **If you check Lab 3 again, note that the variable `y_test` has never been used**, we just inspected the quality of the classification *visually*.

The variable `y_test` contains the true labels of the test set and is the main resource to conduct the **numerical evaluation of the classification model**. All numerical scores start from the [**confusion matrix**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).


In [ ]:
from sklearn.metrics import confusion_matrix

cm_results = confusion_matrix(y_test, y_predicted)
cm_results

In [ ]:
# Visual representation of the same Confusion Matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm_display = ConfusionMatrixDisplay(cm_results).plot()

A lot of metrics for numeric evaluation are inferred from the confusion matrix. It is straightforward with sklearn, and there are many metrics in the [official documentation](https://scikit-learn.org/stable/modules/model_evaluation.html). 

Note that some metrics are more appropriate depending on whether:
- Your dataset is a **binary classification task** or a **multiclass classification task**.
- Your dataset is **balanced** or **imbalanced**

These are technical details to keep in mind in real data science pipelines, but we will continue with the basic metrics for simplicity. Even though note that the bank dataset is imbalanced and is a binary classification task.


In [ ]:
# Individual performance metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

acc_res = accuracy_score(y_test, y_predicted)
prec_res = precision_score(y_test, y_predicted)
reca_res = recall_score(y_test, y_predicted)
f1_res = f1_score(y_test, y_predicted)

print(f"The performance of the DT classifier that we proposed is:")
print(f"\t Accuracy=\t{acc_res}")
print(f"\t Precision=\t{prec_res}")
print(f"\t Recall=\t{reca_res}")
print(f"\t F1-score=\t{f1_res}")

In [ ]:
# Return a full classification score report
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predicted))


### Problem with single train-test partitioning

This single train-test split has a significant problem. There is a fraction of the dataset that is holdout for testing and never goes in the training process. 

#### Reflect ❓
- Is this performance acceptable for a binary classification task and an unbalanced dataset?
- What would happen if the random split puts into the test set a group of data points that are actually very different from the main data samples (they are outliers)?


# 2. Cross-validation (CV)

Cross-validation (CV) is a strategy to solve the problem of single train-test split and **the correct approach** to evaluate models performance, [read more here](https://scikit-learn.org/stable/modules/cross_validation.html), or [here](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html). The CV partitions a dataset into `k` folds or groups. Then, each fold is part of the test set once. It means that a CV creates `k` different classification models instead of only one, and note that it might be very time consuming for classifiers that take a lot of time training or very large datasets.

There are even **better** methods such as [*stratified cross-validation*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) that partitions the group following the same proportion in the target class, or the [*leave-one-out cross-validation*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html) that is suitable for small datasets. We leave these topics out of the scope of these labs, but feel free to use it in the homeworks or in your own projects.

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
CV_indices = kf.split(data_X, data_y)

KFold returns a set of groups representing the indices of the train and the test sets. 

We will use later a simpler approach that calculates the metrics automatically, but **note that** if we iterate over the indices, there are **5 splits** of equal size.

In [ ]:
for train_index, test_index in CV_indices:
    print("TRAIN:", train_index.shape, "TEST:", test_index.shape)

## Model evaluation with automatic CV

To apply CV, we start again from the original dataset in `data_X` and `data_y` before they were partitioned.

Then, we apply a function that returns an evaluation score after conducting CV [`sklearn.model_selection.cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html).

Below, we use an example with the same decision tree created a couple of cells above. We perform a 5-fold cross validation, and specify the scoring metrics with a list of string based on the available [scoring parameters](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter). 

Note that `accuracy` is a *single* number averaged over the confusion matrix, but other metrics like `precision`, `recall` or `f1-score` are computed *per class*. So, in multi-class classification, these metrics need to be averaged over all classes to produce a single number. Sklearn has three types of average: macro-average, micro-average, weighted-average. That is why the scoring metrics below have a suffix. [Read documentation here.](https://scikit-learn.org/stable/modules/model_evaluation.html#from-binary-to-multiclass-and-multilabel)

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
# Define the evaluation metrics to be returned by the cross-validation
scoring_metrics = ["accuracy", "precision_macro", "recall_macro", "f1_macro"] # Metrics of interest

# Execute the cross-validation
results_eval_dt = cross_validate(dt_classifier, data_X, data_y, cv=5, scoring=scoring_metrics)

Each score has a dictionary with the name of the metric and five values corresponding to the 5-folds that we indicated.

In addition, the cross validation gives us the training time (`fit_time`) and prediction time (`score_time`)

In [ ]:
results_eval_dt

In [ ]:
# Averaging the results over the 5-folds and store in a new dictionary
averaged_results_dt = {
                    "classifier_name": "DT"
                    }

# Populate the dictionary with the results of the cross-validation
for metric_name, scores in results_eval_dt.items():
    averaged_results_dt[metric_name] = [scores.mean()]

averaged_results_dt

In [ ]:
# Create a dataframe with the results
results_dt_dataframe = pd.DataFrame(averaged_results_dt)
results_dt_dataframe

# 3. Experimental evaluation of best performance

Now, we have seen how to properly measure the performance of a classifier **numerically**, useful for most classification tasks where the dataset contains many features and the decision boundary cannot be visually inspected.

To finalize, we will perform again an experimental evaluation where we follow the same logic than Lab3, but now we also apply **cross validation** and **evaluation metrics** to assess the best performance.

In this example, we will just work with two types of classifiers with different hyperparameters. But the experimental approach is scalable for as many classifiers and variants as desired.

We will test on the `bank` dataset, the following classifiers:
- Random Forest with 10 estimators
- Random Forest with 100 estimators
- SVM with Linear kernel
- SVM with Gaussian kernel

The procedure will apply 10-fold cross-validation and the scoring metrics are the same than shown above.

In [ ]:
# We will apply the classifiers on the normalized dataset, as recommended for KNN and SVM

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_X_norm = scaler.fit_transform(data_X)

print(f"data_X min. value: {data_X_norm.min()}, max. value: {data_X_norm.max()}")

In [ ]:
# This approach to organize the classifiers is described in Lab 3

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

MODELS_TO_TEST = {
    "RF_10": RandomForestClassifier(n_estimators=10, max_depth=5),
    "RF_100": RandomForestClassifier(n_estimators=100, max_depth=5),
    "SVM_lin": SVC(kernel='linear'),
    "SVM_rbf": SVC(kernel='rbf'),
}

# Define the number of splits 
NUMBER_OF_SPLITS = 10

# Scoring metrics
SCORING_METRICS = ["accuracy", "precision_macro", "recall_macro", "f1_macro"] # Metrics of interest

# Create empty DataFrame to populate  the name of the classifier and the six values returned from `cross_validate()`
results_evaluation = pd.DataFrame({
                                    "classifier_name":[],
                                    "fit_time": [],
                                    "score_time": [],
                                    "test_accuracy": [],
                                    "test_precision_macro": [],
                                    "test_recall_macro": [],
                                    "test_f1_macro": [],
                                    })

In [ ]:
#### ITERATION FOR THE EXPERIMENT

for name, classifier in MODELS_TO_TEST.items():
    
    print(f"Currently training the classifier {name}.")

    # Get the evaluation metrics per fold after cross-validation
    # Note that we are passing the normalized array `data_X_norm` to all classifiers
    scores_cv = cross_validate(classifier, data_X_norm, data_y, cv=NUMBER_OF_SPLITS, scoring=SCORING_METRICS)

    # Average the scores among folds
    dict_this_result = {
                    "classifier_name":[name],
                    }
    # Populate the dictionary with the results of the cross-validation
    for metric_name, score_per_fold in scores_cv.items():
        dict_this_result[metric_name] = [ score_per_fold.mean() ]

    #### Generate the results to populate the pandas.DataFrame
    this_result = pd.DataFrame(dict_this_result)

    # Append to the main dataframe with the results 
    results_evaluation = pd.concat([results_evaluation, this_result], ignore_index=True)

print("The experimental setup has finished")

In [ ]:
results_evaluation

### ❓ - Optional tasks and reflection: 

The labs on classification are officially over. We have finished evaluating multiple classifiers for a single dataset and getting the numerical evaluation metrics.

Below, there are some **optional** extra tasks if you want to continue exploring, but **they are not the HW**:

- Try to visualize the final results using plots as we did in the lab 3.
- Which was the best classifier on average? and the worst?
- Which was the fastest to train? and the fastest to predict? Why?
- *Advanced:* Instead of testing RF with predefined `n_estimators=10` or `100`, try to setup a similar experiment finding the optimal hyperparameter among a set of options (*Hint: Grid Search*).

----

# Homework - Lab 4

The homework consists on submitting the Jupyter notebook started in Lab3, but extended with a numerical evaluation of the classification models on a public dataset of your choice.

**INSTRUCTIONS:**

Continue working on the same jupyter notebook from the Lab 3 where you worked on the following sections:
 1. Choose a dataset for your case study
 2. Create a blank jupyter notebook
 3. Basic EDA
 4. Classification

## 5. Evaluation

Add more cells below to develop the tasks below. You will propose and run an experimental evaluation on your dataset:

1. Use the public dataset from Lab3.
2. Select a subset of features (minimum 3) at your own choice. Remember also to include the **target variable** to run a classification task.
3. Apply ordinal encoding or one-hot encoding to the categorical variables, if you have chosen any.
4. Design an experiment where you apply the three classification models (DT, RF, KNN) from Lab3 over your dataset.
5. The experiment should output, for each classifier, the average accuracy, and macro average of precision, recall, and f1-score. Also apply the mean of the results per fold after cross-validation, as shown in the Lab 4.
6. Make at least two plots where you summarize the experimental results graphically. 
7. *Analyze in a markdown cell:* Which classification model seems to perform better in your data? Would you deploy it in a real-life task? Why or why not?
8. *Final reflection, in a markdown cell:* So far you chose a handful of classifiers with predefined hyperparameters. Describe briefly how do you think you can determine experimentally which hyperparameter performs better for a given classifier? **Note that you do not need to implement this evaluation,** just describe in a couple of sentences the steps or Python functions that you would use for this purpose.

9. **Deliverable:** Send your jupyter notebook `DSHI_Lab3and4_yourname_yourlastname.ipynb` together with the dataset file(s) necessary to execute the notebook. Follow this checklist before submitting your homework:
  * [x] When you think that your Jupyter notebook is finished
  * [ ] Check that your notebook contains the code to generate the plots from Lab3.
  * [ ] Clean the variables in memory by `restarting the kernel/runtime`
  * [ ] Restart the execution of the notebook from the beginning clicking on the option `Run All`
  * [ ] Verify that the notebook runs until the last cell without errors in the middle.
  * [ ] If there are errors, you need to fix them until it runs properly from beginning to end.
  * [ ] Check again that a clean run of your notebook finishes without errors.
  * [ ] Save the notebook (or download it if you are working on Google Colab).
  * [ ] Compress the notebook `.ipynb` and the ***original*** dataset file(s) in a `.zip` file.
  * [ ] Submit the `.zip` file on iLearn

